In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1608775,2021-08-30,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1608776,2021-08-31,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1608777,2021-09-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1608778,2021-09-02,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
42139,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42141,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42143,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42145,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42147,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
981947,2021-08-30,Arkansas,Arkansas,5001,3075,51.00,5000,Arkansas,AR,Arkansas,State,3017804
981949,2021-08-31,Arkansas,Arkansas,5001,3094,52.00,5000,Arkansas,AR,Arkansas,State,3017804
981951,2021-09-01,Arkansas,Arkansas,5001,3109,52.00,5000,Arkansas,AR,Arkansas,State,3017804
981953,2021-09-02,Arkansas,Arkansas,5001,3140,53.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1608775,2021-08-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1608776,2021-08-31,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1608777,2021-09-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1608778,2021-09-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-09-03') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
591,2021-09-03,Snohomish,Washington,51968,655.00,53061,WA,County,822083,6321.50,79.68
1180,2021-09-03,Cook,Illinois,595861,11351.00,17031,IL,County,5150233,11569.59,220.40
1768,2021-09-03,Orange,California,307051,5250.00,6059,CA,County,3175692,9668.79,165.32
2355,2021-09-03,Maricopa,Arizona,648475,10880.00,4013,AZ,County,4485414,14457.42,242.56
2942,2021-09-03,Los Angeles,California,1415563,25401.00,6037,CA,County,10039107,14100.49,253.02
...,...,...,...,...,...,...,...,...,...,...,...
1607599,2021-09-03,Wheeler,Oregon,60,1.00,41069,OR,County,1332,4504.50,75.08
1607925,2021-09-03,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1608220,2021-09-03,Esmeralda,Nevada,42,0.00,32009,NV,County,873,4811.00,0.00
1608511,2021-09-03,Loving,Texas,7,0.00,48301,TX,County,169,4142.01,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
591,2021-09-03,Snohomish,Washington,51968,655.00,53061,WA,County,822083,6321.50,79.68,79.68,6321.50
1180,2021-09-03,Cook,Illinois,595861,11351.00,17031,IL,County,5150233,11569.59,220.40,220.40,11569.59
1768,2021-09-03,Orange,California,307051,5250.00,6059,CA,County,3175692,9668.79,165.32,165.32,9668.79
2355,2021-09-03,Maricopa,Arizona,648475,10880.00,4013,AZ,County,4485414,14457.42,242.56,242.56,14457.42
2942,2021-09-03,Los Angeles,California,1415563,25401.00,6037,CA,County,10039107,14100.49,253.02,253.02,14100.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607599,2021-09-03,Wheeler,Oregon,60,1.00,41069,OR,County,1332,4504.50,75.08,75.08,4504.50
1607925,2021-09-03,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1608220,2021-09-03,Esmeralda,Nevada,42,0.00,32009,NV,County,873,4811.00,0.00,0.00,4811.00
1608511,2021-09-03,Loving,Texas,7,0.00,48301,TX,County,169,4142.01,0.00,0.00,4142.01


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1027016,2021-09-03,Hale,Alabama,2720,81.00,1065,AL,County,14651,18565.29,552.86,552.86,18565.29,1
1118461,2021-09-03,Clarke,Alabama,4280,68.00,1025,AL,County,23622,18118.70,287.87,287.87,18118.70,2
711243,2021-09-03,Etowah,Alabama,17485,394.00,1055,AL,County,102268,17097.23,385.26,385.26,17097.23,3
596376,2021-09-03,Franklin,Alabama,5309,90.00,1059,AL,County,31362,16928.13,286.97,286.97,16928.13,4
802581,2021-09-03,Lowndes,Alabama,1623,55.00,1085,AL,County,9726,16687.23,565.49,565.49,16687.23,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979858,2021-09-03,Converse,Wyoming,1506,24.00,56009,WY,County,13822,10895.67,173.64,173.64,10895.67,19
980383,2021-09-03,Sublette,Wyoming,961,11.00,56035,WY,County,9831,9775.20,111.89,111.89,9775.20,20
1358999,2021-09-03,Niobrara,Wyoming,228,3.00,56027,WY,County,2356,9677.42,127.33,127.33,9677.42,21
1219863,2021-09-03,Lincoln,Wyoming,1893,14.00,56023,WY,County,19830,9546.14,70.60,70.60,9546.14,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
802581,2021-09-03,Lowndes,Alabama,1623,55.00,1085,AL,County,9726,16687.23,565.49,565.49,16687.23,5,1
1027016,2021-09-03,Hale,Alabama,2720,81.00,1065,AL,County,14651,18565.29,552.86,552.86,18565.29,1,2
865818,2021-09-03,Greene,Alabama,1148,40.00,1063,AL,County,8111,14153.62,493.16,493.16,14153.62,42,3
340204,2021-09-03,Walker,Alabama,9359,300.00,1127,AL,County,63521,14733.71,472.28,472.28,14733.71,32,4
710715,2021-09-03,Dallas,Alabama,4386,170.00,1047,AL,County,37196,11791.59,457.04,457.04,11791.59,60,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529389,2021-09-03,Weston,Wyoming,802,6.00,56045,WY,County,6927,11577.88,86.62,86.62,11577.88,17,19
1189242,2021-09-03,Uinta,Wyoming,3053,15.00,56041,WY,County,20226,15094.43,74.16,74.16,15094.43,3,20
1219863,2021-09-03,Lincoln,Wyoming,1893,14.00,56023,WY,County,19830,9546.14,70.60,70.60,9546.14,22,21
799419,2021-09-03,Albany,Wyoming,5291,21.00,56001,WY,County,38880,13608.54,54.01,54.01,13608.54,9,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,79.68,6321.50,24,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,79.68,6321.50,24,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,79.68,6321.50,24,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,79.68,6321.50,24,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,79.68,6321.50,24,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605037,2021-08-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1605038,2021-08-31,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1605039,2021-09-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1605040,2021-09-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
1022756,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,18565.29,2,1,1.00
1022757,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,18565.29,2,1,0.00
1022758,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,18565.29,2,1,0.00
1022759,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,18565.29,2,1,0.00
1022760,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,552.86,18565.29,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240467,2021-08-30,Crook,Wyoming,545,13.00,56011,WY,County,7584,7186.18,171.41,171.41,7568.57,9,23,10.00
1240468,2021-08-31,Crook,Wyoming,551,13.00,56011,WY,County,7584,7265.30,171.41,171.41,7568.57,9,23,6.00
1240469,2021-09-01,Crook,Wyoming,561,13.00,56011,WY,County,7584,7397.15,171.41,171.41,7568.57,9,23,10.00
1240470,2021-09-02,Crook,Wyoming,565,13.00,56011,WY,County,7584,7449.89,171.41,171.41,7568.57,9,23,4.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
799367,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,16687.23,1,5,1.00,0.00
799368,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,16687.23,1,5,0.00,0.00
799369,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,16687.23,1,5,0.00,0.00
799370,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,16687.23,1,5,0.00,0.00
799371,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,16687.23,1,5,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336438,2021-08-30,Teton,Wyoming,4419,11.00,56039,WY,County,23464,18833.11,46.88,46.88,19186.84,23,1,63.00,0.00
336439,2021-08-31,Teton,Wyoming,4427,11.00,56039,WY,County,23464,18867.20,46.88,46.88,19186.84,23,1,8.00,0.00
336440,2021-09-01,Teton,Wyoming,4460,11.00,56039,WY,County,23464,19007.84,46.88,46.88,19186.84,23,1,33.00,0.00
336441,2021-09-02,Teton,Wyoming,4476,11.00,56039,WY,County,23464,19076.03,46.88,46.88,19186.84,23,1,16.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-09-03') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
406388,2021-09-03,Imperial,California,33476,750.00,6025,CA,County,181215,18473.08,413.87,413.87,18473.08,1,2,56.00,1.00
2942,2021-09-03,Los Angeles,California,1415563,25401.00,6037,CA,County,10039107,14100.49,253.02,253.02,14100.49,2,6,2614.00,37.00
201858,2021-09-03,San Bernardino,California,337242,5369.00,6071,CA,County,2180085,15469.21,246.27,246.27,15469.21,3,4,690.00,5.00
713307,2021-09-03,Inyo,California,1589,39.00,6027,CA,County,18039,8808.69,216.20,216.20,8808.69,4,33,20.00,0.00
90025,2021-09-03,Stanislaus,California,76095,1168.00,6099,CA,County,550660,13818.87,212.11,212.11,13818.87,5,9,360.00,9.00
71565,2021-09-03,San Joaquin,California,92693,1540.00,6077,CA,County,762148,12162.07,202.06,202.06,12162.07,6,16,416.00,0.00
41592,2021-09-03,Riverside,California,346375,4759.00,6065,CA,County,2470546,14020.18,192.63,192.63,14020.18,7,8,802.00,10.00
111702,2021-09-03,Tulare,California,67922,872.00,6107,CA,County,466195,14569.44,187.05,187.05,14569.44,8,5,337.00,2.00
646141,2021-09-03,Merced,California,37462,517.00,6047,CA,County,277680,13491.07,186.19,186.19,13491.07,9,10,197.00,4.00
40500,2021-09-03,Fresno,California,128463,1804.00,6019,CA,County,999101,12857.86,180.56,180.56,12857.86,10,13,482.00,4.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1518649,2021-09-03,Lassen,California,6341,27.00,6035,CA,County,30573,20740.52,88.31,88.31,20740.52,37,1,0.00,0.00
406388,2021-09-03,Imperial,California,33476,750.00,6025,CA,County,181215,18473.08,413.87,413.87,18473.08,1,2,56.00,1.00
865760,2021-09-03,Kings,California,28205,266.00,6031,CA,County,152940,18441.87,173.92,173.92,18441.87,11,3,195.00,0.00
201858,2021-09-03,San Bernardino,California,337242,5369.00,6071,CA,County,2180085,15469.21,246.27,246.27,15469.21,3,4,690.00,5.00
111702,2021-09-03,Tulare,California,67922,872.00,6107,CA,County,466195,14569.44,187.05,187.05,14569.44,8,5,337.00,2.00
2942,2021-09-03,Los Angeles,California,1415563,25401.00,6037,CA,County,10039107,14100.49,253.02,253.02,14100.49,2,6,2614.00,37.00
253989,2021-09-03,Kern,California,126859,1482.00,6029,CA,County,900202,14092.28,164.63,164.63,14092.28,14,7,609.00,4.00
41592,2021-09-03,Riverside,California,346375,4759.00,6065,CA,County,2470546,14020.18,192.63,192.63,14020.18,7,8,802.00,10.00
90025,2021-09-03,Stanislaus,California,76095,1168.00,6099,CA,County,550660,13818.87,212.11,212.11,13818.87,5,9,360.00,9.00
646141,2021-09-03,Merced,California,37462,517.00,6047,CA,County,277680,13491.07,186.19,186.19,13491.07,9,10,197.00,4.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
406388,2021-09-03,Imperial,California,33476,750.00,6025,CA,County,181215,18473.08,413.87,413.87,18473.08,1,2,56.00,1.00
2942,2021-09-03,Los Angeles,California,1415563,25401.00,6037,CA,County,10039107,14100.49,253.02,253.02,14100.49,2,6,2614.00,37.00
201858,2021-09-03,San Bernardino,California,337242,5369.00,6071,CA,County,2180085,15469.21,246.27,246.27,15469.21,3,4,690.00,5.00
713307,2021-09-03,Inyo,California,1589,39.00,6027,CA,County,18039,8808.69,216.20,216.20,8808.69,4,33,20.00,0.00
90025,2021-09-03,Stanislaus,California,76095,1168.00,6099,CA,County,550660,13818.87,212.11,212.11,13818.87,5,9,360.00,9.00
71565,2021-09-03,San Joaquin,California,92693,1540.00,6077,CA,County,762148,12162.07,202.06,202.06,12162.07,6,16,416.00,0.00
41592,2021-09-03,Riverside,California,346375,4759.00,6065,CA,County,2470546,14020.18,192.63,192.63,14020.18,7,8,802.00,10.00
111702,2021-09-03,Tulare,California,67922,872.00,6107,CA,County,466195,14569.44,187.05,187.05,14569.44,8,5,337.00,2.00
646141,2021-09-03,Merced,California,37462,517.00,6047,CA,County,277680,13491.07,186.19,186.19,13491.07,9,10,197.00,4.00
40500,2021-09-03,Fresno,California,128463,1804.00,6019,CA,County,999101,12857.86,180.56,180.56,12857.86,10,13,482.00,4.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5342,08/30/21,Lassen,6265,27.00,20491.94,88.31,37,1,26.00,0.00
5343,08/31/21,Lassen,6265,27.00,20491.94,88.31,37,1,0.00,0.00
5344,09/01/21,Lassen,6341,27.00,20740.52,88.31,37,1,76.00,0.00
5345,09/02/21,Lassen,6341,27.00,20740.52,88.31,37,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5428,08/30/21,Fresno,125743,1790.00,12585.61,179.16,10,13,423.00,-2.00
5429,08/31/21,Fresno,126108,1790.00,12622.15,179.16,10,13,365.00,0.00
5430,09/01/21,Fresno,127396,1801.00,12751.06,180.26,10,13,1288.00,11.00
5431,09/02/21,Fresno,127981,1800.00,12809.62,180.16,10,13,585.00,-1.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)